In [13]:
import math
import sys
from scipy.special import logsumexp


def back_log(observed, states, tp, ep):
    # 状態 FとLが格納される
    T = {}
    Ts = []
    # 初期化
    T["F"] = tp["F"]["F"]
    T["L"] = tp["L"]["F"]
    
    Ts.insert(0,T)

    # 再帰
    for ob in observed[:-1]:
        T = back_l(ob, states, T, tp, ep)
        Ts.insert(0,T)
        
        
    # 終了処理
    U = {}
    
    l = [tp["F"]["F"]+T["F"]+ep["F"][observed[-1]],tp["F"]["L"]+T["L"]+ep["L"][observed[-1]]]
    U["F"] = logsumexp(l)
    
    l = [tp["L"]["F"]+T["F"]+ep["F"][observed[-1]], tp["L"]["L"]+T["L"]+ep["L"][observed[-1]]]
    U["L"] = logsumexp(l)

    return Ts,U


def back_l(ob, states, T, tp, ep):
    # 状態 FとLが格納される
    U = {}
    
    l = [tp["F"]["F"]+T["F"]+ep["F"][ob],tp["F"]["L"]+T["L"]+ep["L"][ob]]
    U["F"] = logsumexp(l)
    
    l = [tp["L"]["F"]+T["F"]+ep["F"][ob], tp["L"]["L"]+T["L"]+ep["L"][ob]]
    U["L"] = logsumexp(l)

    return U


def back_scaling(observed, states, sp, tp, ep):
    T = {}
    Ts = []
    P = 1
    
    # 初期化
    T["F"] = tp["F"]["F"]
    T["L"] = tp["L"]["F"]
    
    Ts.insert(0,T)
        
    # 再帰
    for ob in observed[:-1]:
        T,si = back_s(ob, states, T, tp, ep)
        P *= si
        Ts.insert(0,T)
        
    # 終了処理
    si = 0
    S = {}
    
    S["F"] = tp["F"]["F"]*T["F"]*ep["F"][ob]+tp["F"]["L"]*T["L"]*ep["L"][ob]
    S["L"] = tp["L"]["F"]*T["F"]*ep["F"][ob]+tp["L"]["L"]*T["L"]*ep["L"][ob]
    si = S["F"]+S["L"]
    
    P *= si

    return Ts,P

def back_s(ob, states, T, tp, ep):
    # 状態 FとLが格納される
    U = {}
    si = 0
    S = {}
    
    
    S["F"] = tp["F"]["F"]*T["F"]*ep["F"][ob]+tp["F"]["L"]*T["L"]*ep["L"][ob]
    S["L"] = tp["L"]["F"]*T["F"]*ep["F"][ob]+tp["L"]["L"]*T["L"]*ep["L"][ob]
    
    si = S["F"]+S["L"]
    
    U["F"] = (1/si)*S["F"]
    U["L"] = (1/si)*S["L"]

    return U,si

In [15]:
f = open('observed.txt')
observed = f.read()
f.close()

observed = ''.join(list(reversed(observed)))


###########  log
states = ("F", "L")
transit_prob = {"F": {"F": math.log(0.95), "L": math.log(0.05)},
                "L": {"F": math.log(0.1), "L": math.log(0.9)}}
emission_prob = {'F': {'1': math.log(1/6), '2': math.log(1/6), '3': math.log(1/6), '4': math.log(1/6), '5': math.log(1/6), '6': math.log(1/6)},
                 'L': {'1': math.log(1/10), '2': math.log(1/10), '3': math.log(1/10), '4': math.log(1/10), '5': math.log(1/10), '6': math.log(1/2)}}

_,logP = back_log(observed, states, transit_prob, emission_prob)

print("周辺化確率:{}".format(logP))



###########scale
states = ("F", "L")
start_prob = {"F": 0.5, "L": 0.5}
transit_prob = {"F": {"F": 0.95, "L": 0.05},
                "L": {"F": 0.1, "L": 0.9}}
emission_prob = {'F': {'1': 1/6, '2': 1/6, '3': 1/6, '4': 1/6, '5': 1/6, '6': 1/6},
                 'L': {'1': 1/10, '2': 1/10, '3': 1/10, '4': 1/10, '5': 1/10, '6': 1/2}}


_,P = back_scaling(observed, states,
                          start_prob, transit_prob, emission_prob)

print("周辺化確率:{}".format(math.log(P)))

周辺化確率:{'F': -516.6013987084424, 'L': -517.7315431779313}
周辺化確率:-516.3215069583993
